In [2]:

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
import torch.nn.functional as F
import time

# Dataset

In [1]:
from load_dataset import download_dataset
from load_model import create_classification_model


In [231]:
trainloader,testloader = download_dataset(batch_size = 128)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = create_classification_model()

In [ ]:
%load 

In [268]:
%%writefile load_model.py
import torch
from torch import nn
from torchvision.models import resnet50, ResNet50_Weights

def download_model():
    
    weights = ResNet50_Weights.IMAGENET1K_V2
    model = resnet50(weights=weights)
    
    return model

def create_teacher_model(n_classes=10):
    
    model = download_model()
    classification_model = Resnet_Classification(model,n_classes)
    
    
    return classification_model


def create_student_model(n_classes=10):
    
    model = Student(n_classes)
    
    return model

class Resnet_Classification(nn.Module):
    
    def __init__(self,base_model,n_classes):
        super().__init__()
        self.base_model = base_model
        self.n_classes = n_classes
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(1000,n_classes)
        
    def forward(self,inputs,labels = None,**kargs):
        outputs = self.base_model(inputs,**kargs)
        x = self.dropout(outputs)
        logits = self.classifier(x)
        loss = None
        out = {} ## Output dictionary
        out['logits'] = logits
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1,self.n_classes),labels.view(-1))
            out["loss"] = loss
           
        return out

    
    

class Student(nn.Module):
    
    def __init__(self,n_classes):
        super().__init__()
        self.n_classes = n_classes
        self.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.batchnorm = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        self.conv2 = nn.Conv2d(64, 8, kernel_size=(3, 3), stride=(1, 1), bias=False)
        self.classifier = nn.Linear(288,self.n_classes)
    
    def distillation_loss(self,logits,labels,teacher_logits,T,alpha):
    
        train_loss_fct = nn.CrossEntropyLoss() 
        train_loss = train_loss_fct(logits,labels) ## Training loss

        distill_loss_fnt =  nn.KLDivLoss() ## Distillation loss

        distill_loss = distill_loss_fnt(F.log_softmax(logits/T,dim=1),F.log_softmax(teacher_logits/T,dim=1))

        ## Total loss
        loss = T*T*0.9*distill_loss+(1-0.9)*train_loss

        return loss


    def forward(self,x,teacher_logits = None,labels=None,T=5,alpha=0.5):
        x = self.conv1(x)
        x = self.batchnorm(x)
        x = nn.functional.relu(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.Flatten()(x)
        logits = self.classifier(x)
        
        
        
        out = {}
        out['logits'] = logits
        
        
       
        if teacher_logits is not None and labels is not None:
            
            out['loss'] = self.distillation_loss(logits,labels,teacher_logits,T,alpha)
            
            return out
        
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            train_loss = loss_fct(logits,labels)
            out['loss'] = train_loss
        
   
        return out

Writing load_model.py


In [267]:
%%writefile train.py

from load_dataset import download_dataset
from load_model import create_classification_model




def train(classification_model,trainloader,testloader,optimizer,steps = 100,teacher_logits=None):
    
    
    train_step = 0
    running_loss = []
    while True:
        for batch_count,(batch,labels) in enumerate(trainloader):
            if train_step>steps:
                
                return 
            
            if teacher_logits:
                
                teacher_logit = teacher_logits[batch_count]
                    
                output_dict = classification_model(batch,teacher_logits=teacher_logit,labels=labels)
            else:
                output_dict = classification_model(batch,labels=labels)
                
            loss = output_dict['loss']
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss.append(loss.item())
           
            if train_step%100 == 0:
                print(f"Step:{train_step}. Train Loss {sum(running_loss)/len(running_loss)}")
                
                running_loss = []
            
            if train_step%500 == 0:
                
                acc,test_loss = test(classification_model,testloader)
                print(f"Step:{train_step}. Validation loss: {test_loss}. Validation Accuracy: {acc}")
        
            train_step+=1
            
            
def test(classification_model,testloader):
    
    test_loss,correct = 0,0
    for batch,labels in testloader:
        
        with torch.no_grad():
            outputs = classification_model(batch,labels=labels)
            logits = outputs['logits']
            preds = torch.argmax(logits,axis=-1)
            test_loss += outputs['loss'].item()
            correct += (preds==labels).sum().item()
    
    
    acc = correct/len(testloader.dataset)
    test_loss = test_loss/len(testloader)
    
    return acc,test_loss

if __name__ == "__main__":
    
    
    trainloader,testloader = download_dataset()
    
    teacher_model = create_classification_model()
    

Writing train.py


In [260]:
for param in model.parameters():
    param.requires_grad = True

In [180]:
optimizer = torch.optim.Adam(params=model.parameters(),lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [241]:
from tqdm import tqdm
teacher_logits = []
for batch,_ in tqdm(trainloader):
    teacher_logits.append(predict(model,batch)['logits'])

  0%|▍                                                                                                                                                                      | 1/391 [00:08<54:08,  8.33s/it]

Time taken 5.068665027618408


  1%|▊                                                                                                                                                                      | 2/391 [00:13<41:42,  6.43s/it]

Time taken 5.101564884185791


  1%|█▎                                                                                                                                                                     | 3/391 [00:18<37:41,  5.83s/it]

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 390/391 [34:29<00:05,  5.03s/it]

Time taken 5.0067298412323


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [34:32<00:00,  4.51s/it]

Time taken 3.28872013092041


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [34:42<00:00,  5.33s/it]


In [ ]:
for p

In [261]:
train(model,trainloader,testloader,optimizer,steps=1000)

Step:0. Train Loss 0.41296520829200745
Step:0. Validation loss: 0.5910269825518886. Validation Accuracy: 0.827
Step:100. Train Loss 0.46224588707089426
Step:200. Train Loss 0.44608253449201585
Step:300. Train Loss 0.41712217807769775
Step:400. Train Loss 0.9450620767474175
Step:500. Train Loss 1.1603494328260422
Step:500. Validation loss: 0.8606878285166584. Validation Accuracy: 0.7267


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x1148990d0>
Traceback (most recent call last):
  File "/Users/hung/miniforge3/envs/bourbon/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/Users/hung/miniforge3/envs/bourbon/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1474, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Users/hung/miniforge3/envs/bourbon/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/Users/hung/miniforge3/envs/bourbon/lib/python3.8/multiprocessing/popen_fork.py", line 44, in wait
    if not wait([self.sentinel], timeout):
  File "/Users/hung/miniforge3/envs/bourbon/lib/python3.8/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/Users/hung/miniforge3/envs/bourbon/lib/python3.8/selectors.py", line 415, in select
    

KeyboardInterrupt: 

In [243]:
student_model = Student(10)

In [244]:
student_opt = torch.optim.Adam(params=student_model.parameters(),lr=1e-3)


In [252]:
train(student_model,trainloader,testloader,student_opt,steps=5000)

Step:0. Train Loss 0.7889266610145569
Step:0. Validation loss: 1.0580933207198033. Validation Accuracy: 0.627
Step:100. Train Loss 0.9727080065011978
Step:200. Train Loss 0.9813728594779968
Step:300. Train Loss 1.0129813933372498
Step:400. Train Loss 0.9921359550952912
Step:500. Train Loss 0.9745008969306945
Step:500. Validation loss: 1.0568674463260024. Validation Accuracy: 0.6361
Step:600. Train Loss 0.9770104998350143
Step:700. Train Loss 0.9855066519975663
Step:800. Train Loss 0.9667122340202332
Step:900. Train Loss 0.9856679660081863
Step:1000. Train Loss 0.9851827013492584
Step:1000. Validation loss: 1.078304808351058. Validation Accuracy: 0.6292
Step:1100. Train Loss 0.9646671545505524
Step:1200. Train Loss 0.9871096676588058
Step:1300. Train Loss 0.9795529127120972
Step:1400. Train Loss 0.9747875857353211
Step:1500. Train Loss 0.9577085196971893
Step:1500. Validation loss: 1.0452931251706956. Validation Accuracy: 0.638
Step:1600. Train Loss 0.9664068067073822
Step:1700. Train L

In [246]:
student_model_distill = Student(10)
student_opt_distill = torch.optim.Adam(params=student_model_distill.parameters(),lr=1e-3)


In [258]:
train(student_model_distill,trainloader,testloader,student_opt_distill,steps=2500,teacher_logits=teacher_logits)

Step:0. Train Loss 0.06987519562244415
Step:0. Validation loss: 1.0010248664059216. Validation Accuracy: 0.6542
Step:100. Train Loss 0.08398781444877386
Step:200. Train Loss 0.08447678543627263
Step:300. Train Loss 0.08440859839320183
Step:400. Train Loss 0.08577650003135204
Step:500. Train Loss 0.08460785366594792
Step:500. Validation loss: 1.013599473464338. Validation Accuracy: 0.6549
Step:600. Train Loss 0.08441449768841267
Step:700. Train Loss 0.08601541992276907
Step:800. Train Loss 0.08517735738307237
Step:900. Train Loss 0.08541839994490147
Step:1000. Train Loss 0.08511019174009561
Step:1000. Validation loss: 1.0093797173681138. Validation Accuracy: 0.6557
Step:1100. Train Loss 0.08470616422593594
Step:1200. Train Loss 0.08378019031137228
Step:1300. Train Loss 0.08416977971792221
Step:1400. Train Loss 0.08613279845565558
Step:1500. Train Loss 0.08508556980639696
Step:1500. Validation loss: 1.0116978280151947. Validation Accuracy: 0.6574
Step:1600. Train Loss 0.08573085896670818

In [138]:
@torch.no_grad()
def predict(model,data):
    start = time.time()
    out = model(data)
    end = time.time()
    print(f"Time taken {end-start}")
    return out